<a href="https://colab.research.google.com/github/BRBrum/Halliburton/blob/main/EXCEL_TO_LAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install lasio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 1.9 MB/s eta 0:00:00


In [3]:
# Este script le o arquivo em formato Excel da ANP, com dados geoquimicios, e
# transforma o formato para aquivo digital de poço em formato .las.

import pandas as pd
import lasio
import os

# Caminhos de entrada e saída (inserir o path do arquivo de entrada)
input_file_path = r'/content/GQ1_Bacia_Paranß_COTP_Area_Blocos 2.xlsx'
# Inserir o oloca que será gravado os arquivos de saíd.
output_directory = r'/content/'

# Carregar o arquivo Excel
df = pd.read_excel(input_file_path, usecols="A,E:O")

# Preencher os valores de profundidade e atributos para cada poço
df['Poço'] = df['Nome Poço ANP'].ffill()  # Preencher valores faltantes com o nome do poço anterior

# Separar os dados por poço, excluindo a coluna 'Poço'
dados_pocos = {poço: grupo.drop(columns=['Nome Poço ANP', 'Poço']).reset_index(drop=True)
               for poço, grupo in df.groupby('Poço')}

# Função para criar um arquivo LAS a partir dos dados do poço
def criar_las(poco_id, dados):
    las = lasio.LASFile()

    # Adicionar informações do cabeçalho
    las.well['WELL'].value = poco_id
    las.well['COMP'].value = 'Companhia de Petróleo'
    las.well['LOC'].value = 'Localização'
    las.well['PROV'].value = 'Bacia'

    # Adicionar dados de profundidade
    las.append_curve('DEPTH', dados.iloc[:, 0].values, unit='m', descr='Profundidade (m)')

    # Adicionar dados dos atributos, exceto 'Poço'
    for col in dados.columns[1:]:
        las.append_curve(col, dados[col].values)

    # Salvar o arquivo LAS no diretório especificado
    output_file_path = os.path.join(output_directory, f'{poco_id}.las')
    las.write(output_file_path)

# Gerar arquivos LAS para cada poço
for poco_id, dados in dados_pocos.items():
    criar_las(poco_id, dados)

print("Arquivos LAS gerados com sucesso!")

Arquivos LAS gerados com sucesso!
